In [1]:
import pandas as pd

In [2]:
## Run this in jupyter so any changes you make to the class will appear just by re-importing
## instead of having to re-start the kernel
%load_ext autoreload
%autoreload 2

In [3]:
from electrical_model import ElectricModel

In [4]:
### read in commuter_df. feed that and the ev file location to class
commuter_df = pd.read_csv("./samples/assignment_00_random.csv",index_col=0)
em_1 = ElectricModel(commuter_df,ev_reference_table_loc="EV_reference_table.csv")
em_1

In [5]:
print(em_1.commuter_df.shape)

(23978, 43)


In [6]:
### read_electric_info_in reads ev_ref_table and creates floor variables
em_1.read_electric_info_in()
print(f"Subway floor kwh: {em_1.subway_floor} || Commuter Rail floor kwh: {em_1.commuterRail_floor}")
em_1.ev_reference_table.head(2)

ev reference table read in. subway floor and commuter rail floor extracted from table
Subway floor kwh: 55240 || Commuter Rail floor kwh: 9147


,TransMode,ChargingPower_kW,HoursToFullyRecharge_hr,Floor_kWh,Multiplier,MultiplierUnit,excludingfromMNY,NumberOfEV,TotalEnergyPerEV,HourEnergyPerEV,Energy
0,AutoOccupants,12.0,7.0,NaN,0.2134,kwh/km,Live outside of MNY & Owner,PERWT,DISTANCE_KM*Multiplier*2,no delay + related to charging time,In
1,Bus,163.0,3.0,NaN,1.7610,kwh/km,Live outside of MNY,PERWT/20,DISTANCE_KM*Multiplier,6 hr delay + related to charging time,In+Out


In [7]:
## clean_commuter_df:
    # creates the MNY_RES column
    # renames the Rand Assignment to TransMode (if necessary)
    # Creates leave_work_hour column
    # Filters out reverse commuters
    # runs excludingfromMNY function

em_1.clean_commuter_df()
print(em_1.commuter_df.shape)

(18870, 46)


In [8]:
em_1.create_in_out_flow_dfs()
em_1.flow_df.head(5)

,HOUR,TransMode,DISTANCE_KM,PERWT,FLOW_DIR
0,9,No option,170.50491,23.0,IN
1,8,Motorcycle,170.50491,48.0,IN
2,8,Motorcycle,170.50491,54.0,IN
3,14,No option,170.50491,29.0,IN
4,9,Motorcycle,170.50491,33.0,IN


In [9]:
# aggregate_data() runs HourEnergyByMode wrapper function
em_1.aggregate_data()

In [10]:
em_1.df_aggregate

### this looks almost identical to the previous notebook's df_agg
### slight differences could be due to the random function 
    # filtering different bicycle and escooter home owners

,Charge_Hour,TransMode,FLOW_DIR,Energy
0,0,AutoOccupants,IN,90.988519
1,0,Bicycle,IN,5367.594135
2,0,Bus,IN,628.909793
3,0,Bus,OUT,8186.082715
4,0,CommuterRail,OUT,9642.600000
...,...,...,...,...
431,23,CommuterRail,ALL,19145.760000
432,23,Ferry,ALL,56.600000
433,23,Motorcycle,ALL,12594.462129
434,23,Subway,ALL,115838.782000


In [11]:
#### Checks from Jingrong's notebook
print(em_1.df_aggregate.TransMode.unique())
print()
print(em_1.df_aggregate.FLOW_DIR.unique())

['AutoOccupants', 'Bicycle', 'Bus', 'CommuterRail', 'Escooter', 'Ferry', 'Motorcycle', 'Subway', 'Taxicab']
Categories (9, object): ['AutoOccupants' < 'Bus' < 'Ferry' < 'Escooter' ... 'Motorcycle' < 'Taxicab' < 'Subway' < 'CommuterRail']

['IN' 'OUT' 'ALL']


In [12]:
## ToDo:
# Create visualization methods 
# Create diagnostic methods that can spit out certain values on demand
# Create input for electrical capacity parameters (once Robert gives us info)